In [1]:
import numpy as np 
import pandas as pd
import time
import datetime
import gc
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from gensim.models import word2vec
import Levenshtein


In [14]:
keng_valid_29 = pd.read_csv('../result/valid_29_pred.csv')
nn_valid_26 = pd.read_csv('../result/submission4.csv')


In [15]:
valid_26_df = keng_valid_29.copy()
valid_26_df.rename(columns={'predicted_score':'keng_pred'}, inplace=True)
valid_26_df['nn_pred'] = nn_valid_26['score'][:len(valid_26_df)]



In [16]:
print(len(nn_valid_26))
print(np.mean(valid_26_df[valid_26_df.is_prefix_in_train == 1]['nn_pred']))
print(np.mean(valid_26_df[valid_26_df.is_prefix_in_train == 0]['nn_pred']))


100000
0.3729864797753706
0.38116897249021026


In [23]:
#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.keng_pred.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

keng_valid_df_after = resultAdjustment(valid_26_df, -0.1121)


0.37251355671061986


In [24]:
valid_26_df['keng_pred'] = keng_valid_df_after


In [28]:
#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.nn_pred.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

nn_valid_df_after0 = resultAdjustment(valid_26_df[valid_26_df.is_prefix_in_train == 0], -0.0571)



0.3724193617292467


In [29]:
valid_26_df.loc[valid_26_df.is_prefix_in_train == 0, 'nn_pred'] = nn_valid_df_after0


In [30]:
print(np.mean(valid_26_df[valid_26_df.is_prefix_in_train == 1]['nn_pred']))
print(np.mean(valid_26_df[valid_26_df.is_prefix_in_train == 0]['nn_pred']))


0.3729864797753706
0.3724193617292467


In [37]:
valid_26 = pd.read_table('../data/oppo_round1_vali_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
valid_26_df['label'] = valid_26['label']


In [38]:
yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 500, 2):
    real_yuzhi = yuzhi / 1000
    valid_26_df['predicted_label'] = valid_26_df['keng_pred'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_26_df['label'], valid_26_df['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)


{'0.35': 0.7388241070332638, '0.352': 0.738685556867375, '0.354': 0.7391947898164595, '0.356': 0.7388273680047459, '0.358': 0.738801168143345, '0.36': 0.7390150105445975, '0.362': 0.7394674085850558, '0.364': 0.7395991534918461, '0.366': 0.7391315192178186, '0.368': 0.7395453409942543, '0.37': 0.73906191369606, '0.372': 0.7389461540390742, '0.374': 0.7391074795725959, '0.376': 0.7397536337758521, '0.378': 0.7395954194622408, '0.38': 0.7394949494949495, '0.382': 0.7395382942678702, '0.384': 0.7392978367698466, '0.386': 0.739692682184695, '0.388': 0.74006908462867, '0.39': 0.7401622872841046, '0.392': 0.7400254777070063, '0.394': 0.7400581959262853, '0.396': 0.739895630819605, '0.398': 0.7398861713582525, '0.4': 0.7395670150748607, '0.402': 0.7400490132851799, '0.404': 0.7400459912668268, '0.406': 0.7401366884125505, '0.408': 0.7405659153979821, '0.41': 0.7403271877434433, '0.412': 0.7405036944531169, '0.414': 0.7403064415259537, '0.416': 0.7405183889744459, '0.418': 0.7402505688802865, 

In [43]:
valid_26_df['predicted_score'] = valid_26_df['keng_pred'] * 0.9 + valid_26_df['nn_pred'] *0.1


In [44]:
yuzhi_dict = {}
# 定义搜索方法获取最佳F1对应的阈值
for yuzhi in range(350, 500, 2):
    real_yuzhi = yuzhi / 1000
    valid_26_df['predicted_label'] = valid_26_df['predicted_score'].map(lambda x : 1 if x > real_yuzhi else 0)
    f1 = f1_score(valid_26_df['label'], valid_26_df['predicted_label'])
    yuzhi_dict[str(real_yuzhi)] = f1
print(yuzhi_dict)


{'0.35': 0.7370967741935482, '0.352': 0.7371515359365048, '0.354': 0.7377725397760754, '0.356': 0.7378339812035626, '0.358': 0.738448641451748, '0.36': 0.7383771225190935, '0.362': 0.738280959809821, '0.364': 0.738770040204497, '0.366': 0.7393996667578523, '0.368': 0.739588785046729, '0.37': 0.7394420718763267, '0.372': 0.7397897897897897, '0.374': 0.7397658619738787, '0.376': 0.7401135621325562, '0.378': 0.7399813606710158, '0.38': 0.7398899379007421, '0.382': 0.7401766060268705, '0.384': 0.7402916930881421, '0.386': 0.740554284261378, '0.388': 0.7403914772148028, '0.39': 0.7402789842113331, '0.392': 0.740427927927928, '0.394': 0.7406438373733487, '0.396': 0.7405693126591066, '0.398': 0.7399943303352833, '0.4': 0.7400196250581005, '0.402': 0.7399265504577666, '0.404': 0.7395371738060069, '0.406': 0.7395154379495701, '0.408': 0.7391700273188501, '0.41': 0.7388946819603756, '0.412': 0.738865815113758, '0.414': 0.7384840871021776, '0.416': 0.7385034869697447, '0.418': 0.7382165939782461,